# Imports and Constants

In [ ]:
%cd /home/caitsithx/courses/deeplearning1/nbs

%matplotlib inline
import utils; reload(utils)
from utils import *
import glob
import numpy as np
import os
from shutil import copyfile
from PIL import Image
from IPython.display import FileLink
import scipy
from vgg16 import Vgg16

In [ ]:
path = "/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring"
trn_path = path + "/train"
vld_path = path + "/valid"
result_path=path + "/result/"
nb_iter = 100

# Todos

1. create validation set and sample
2. move to separate dirs for each set
3. finetune and train
4. submit

# Data Preparation

In [ ]:
%cd /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/0
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/1

labels = np.loadtxt(path + "/train_labels.csv",delimiter=',')
labels[:5]

for label in labels: 
    if label[1] == 0:
        copyfile(trn_path+"/"+str(int(label[0]))+".jpg", trn_path+"/0/"+str(int(label[0]))+".jpg")
    elif label[1] == 1:
        copyfile(trn_path+"/"+str(int(label[0]))+".jpg", trn_path+"/1/"+str(int(label[0]))+".jpg")

In [ ]:
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/valid
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/valid/1
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/valid/0

%cd /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/1
g = glob.glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(200): os.rename(shuf[i], "../../valid/1/" + shuf[i])
    
%cd /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/0
g = glob.glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(200): os.rename(shuf[i], "../../valid/0/" + shuf[i])

In [ ]:
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/sample
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/sample/test
%mkdir /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/sample/test/unknown
%cd /home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/test

g = glob.glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], "../sample/test/unknown/" + shuf[i])

# Fine-tuning and Training

In [ ]:
vgg16 = Vgg16()

In [ ]:
trn_batch = vgg16.get_batches(path+"/train", batch_size=100)
vld_batch = vgg16.get_batches(path+"/valid", batch_size=100)

In [ ]:
vgg16.finetune(trn_batch)

In [ ]:
vgg16.model.optimizer.lr = 0.0005

In [ ]:
weight_file_name = None
for epoch in range(nb_iter):
    print 'iteration %d' % epoch
    vgg16.fit(trn_batch, vld_batch, nb_epoch=1)
    weight_file_name = 'ft%d.h5' % epoch
    vgg16.model.save_weights(result_path+weight_file_name)

# Test

In [ ]:
weight_file_name = 'ft%d.h5' % (nb_iter - 1)
vgg16.model.load_weights(result_path+weight_file_name)

In [ ]:
batches, preds = vgg16.test(path+"/test", batch_size=50)

In [ ]:
print preds[:5]
print batches.filenames[:5]

In [ ]:
Image.open(path+"/test/" + batches.filenames[0])

In [ ]:
save_array(result_path + 'test_preds.dat', preds)
save_array(result_path + 'filenames.dat', batches.filenames)

# submission

In [ ]:
preds = load_array(result_path + 'test_preds.dat')
fileNames = load_array(result_path + 'filenames.dat')

In [ ]:
preds[:5]

In [ ]:
def covert(val):
    if(val > 0.5):
        return 1;
    else:
        return 0;
    

In [ ]:
isInv = [covert(pred) for pred in preds[:,1]]

In [ ]:
isInv[:5]

In [ ]:
ids = [int(fn[8:-4]) for fn in fileNames]

In [ ]:
ids[:5]

In [ ]:
subm = np.stack([ids,isInv], axis=1)

In [ ]:
subm[:5]

In [ ]:
np.savetxt(result_path + "inva_submit.csv", subm, fmt='%d,%.5f', header='name,invasive',comments='')

In [ ]:
%cd /home/caitsithx/courses/deeplearning1/nbs
FileLink("data/invasive-species-monitoring/result/inva_submit.csv")